---
title: "Global Insights Dashboard: Suicide, Happiness, and Internet Use"
author: "my_project"
format: dashboard
theme: sketchy
---

In [ ]:
import pandas as pd
import plotly.express as px
import itables 
import country_converter as cc
from functools import reduce

In [ ]:
#| output: false
# happiness_score dataset
happiness_data = pd.read_csv("data\hapiscore_whr.csv")

happiness_data

In [ ]:
#| output: false
# changing dataset to a long format
happiness_data_long = happiness_data.melt(id_vars="country")
happiness_data_long

In [ ]:
#| output: false
# changing to representative column names
happiness_data_long = happiness_data.melt(id_vars="country", var_name="year", value_name = "percent_happiness").sort_values(["country", "year"])

happiness_data_long

In [ ]:
#| output: false
# checking datatype
happiness_data_long.dtypes

In [ ]:
#| output: false
# converting the 'year' column to integer type, and coercing errors to NaN
happiness_data_long['year'] = pd.to_numeric(happiness_data_long['year'], errors='coerce')

# Confirm the conversion
print(happiness_data_long['year'].dtype)

In [ ]:
#| output: false
# filtering for year 2017 to 2022
happiness_data_long = happiness_data_long[
    (happiness_data_long['year'] >= 2017) & (happiness_data_long['year'] <= 2022)
]

# Check the result
print(happiness_data_long['year'].unique())
print(happiness_data_long["country"].unique())
print(happiness_data_long.shape)

In [ ]:
#| output: false
# Updating rows where 'country' equals 'UAE'
happiness_data_long.loc[happiness_data_long['country'] == 'UAE', 'country'] = 'United Arab Emirates'

# Verify the change
print(happiness_data_long['country'].unique())

In [ ]:
#| output: false

# converting country names to ISO3 codes
happiness_data_long["country_code"] = cc.convert(happiness_data_long["country"], to="ISO3")

# Verify the result
print(happiness_data_long.head())
print(f"Shape after dropping invalid countries: {happiness_data_long.shape}")

In [ ]:
#| output: false
# suicide_rate dataset
suicide_rate = pd.read_csv("data\suicide_rate_data.csv")

suicide_rate

In [ ]:
#| output: false
# selecting applicable columns
suicide_rate_subset = suicide_rate [["Reference area", "Sex", "TIME_PERIOD", "OBS_VALUE"]]

suicide_rate_subset

In [ ]:
#| output: false
# changing to representative column names
suicide_rate_subset = suicide_rate_subset.rename(columns={"Reference area": "country", "Sex": "sex", "TIME_PERIOD": "year", "OBS_VALUE": "mortality_count"})

suicide_rate_subset = suicide_rate_subset.sort_values(["country", "year"])

suicide_rate_subset

In [ ]:
#| output: false
# dropping rows with 'OECD' or 'European Union' in the 'country' column 
# as only interested in single country names
suicide_rate_subset = suicide_rate_subset[
    ~suicide_rate_subset['country'].str.contains("OECD|European Union", case=False, na=False,regex=True)
]

# Verify the result
print(suicide_rate_subset['country'].unique())
print(f"Shape after dropping OECD and European Union rows: {suicide_rate_subset.shape}")

In [ ]:
#| output: false

# unique number of countries in happiness dataset
suic_country_count = suicide_rate_subset["country"].nunique()

suic_country_count

In [ ]:
#| output: false
# internet_use dataset
internet_use = pd.read_csv ("data\household_internet_use.csv")

internet_use 

In [ ]:
#| output: false
# selecting applicable columns
internet_use_subset = internet_use[["Reference area", "TIME_PERIOD", "OBS_VALUE"]]

internet_use_subset

In [ ]:
#| output: false
# changing to representative column names
internet_use_subset = internet_use_subset.rename(columns={"Reference area": "country", "TIME_PERIOD": "year", "OBS_VALUE": "percent_internet_household"})

internet_use_subset

In [ ]:
#| output: false
# dropping rows with 'OECD' or 'European Union' in the 'country' column
internet_use_subset = internet_use_subset[
    ~internet_use_subset['country'].str.contains("OECD|European Union", case=False, na=False,regex=True)
]

# Verify the result
print(internet_use_subset['country'].unique())
print(f"Shape after dropping OECD and European Union rows: {internet_use_subset.shape}")

In [ ]:
#| output: false
# rounding to 1 decimal for consistency with other datasets
internet_use_subset["percent_internet_household"] = internet_use_subset["percent_internet_household"].round(1)

internet_use_subset

In [ ]:
#| output: false
internet_use_subset["country_code"]  = cc.convert(internet_use_subset["country"], to="ISO3")


# Verify the result
print(internet_use_subset.head())
print(f"New shape of the DataFrame: {internet_use_subset.shape}")

In [ ]:
#| output: false
internet_use_subset = internet_use_subset.sort_values(["country","year"])

In [ ]:
#| output: false
#calculating missing values for each dataset
happiness_missing = happiness_data_long.isna().sum()
internet_missing = internet_use_subset.isna().sum()
suicide_missing = suicide_rate_subset.isna().sum()


# combining the results into a single DataFrame
missing_summary = pd.DataFrame({
   'dataset': ['percent_happiness', 'internet_use', 'suicide_rate'],
   'country': [happiness_missing['country'], internet_missing['country'], suicide_missing['country']],
   'year': [happiness_missing['year'], internet_missing['year'], suicide_missing['year']],
   'value_column': [happiness_missing['percent_happiness'], internet_missing['percent_internet_household'], suicide_missing['mortality_count']]
})

missing_summary                      

In [ ]:
#| output: false
# reshaping with melt and naming columns for clarity
missing_summary = missing_summary.melt(
    id_vars='dataset', var_name='dataset_columns', value_name='missing_count'
).sort_values('dataset')
missing_summary

In [ ]:
#| output: false
# bar chart for missing values
fig = px.bar(
    missing_summary,
    x='dataset_columns',
    y='missing_count',
    color='dataset',
    title='Missing Values per Column Across Datasets',
    labels={'missing_count': 'Count of Missing Values', 'dataset_columns': 'Columns'},
    barmode='group'
)

fig.show()

In [ ]:
#| output: false
# dropping missing value from dataset
happiness_data_long.dropna(inplace=True)

In [ ]:
#| output: false
# function to check any persistent missing values
def check_missing_data(df, name):
    print(f"Missing Data Summary for {name}:")
    print(df.isna().sum())
    print("-" * 40)

# Check for missing values in each dataset
check_missing_data(happiness_data_long, "Happiness Data")
check_missing_data(internet_use_subset, "Internet Use Data")
check_missing_data(suicide_rate_subset, "Suicide Rate Data")

In [ ]:
#| output: false

# unique number of countries in happiness dataset
happy_countries = happiness_data_long["country"].nunique()

happy_countries

In [ ]:
#| output: false
# creating a pivoted version of the 'suicide_rate_subset' dataFrame 
# to analyze mortality rates by sex (Male, Female, Total)
# without modifying the original 'suicide_rate_subset
# pivoting the data so each sex becomes its own column
suicide_rate_pivot = suicide_rate_subset.pivot_table(
    index=['country', 'year'],
    columns='sex',
    values='mortality_count',
    aggfunc='sum'  # Aggregate if duplicates exist
).reset_index()

# renaming the columns for clarity
suicide_rate_pivot.columns.name = None  # removing column index name
suicide_rate_pivot = suicide_rate_pivot.rename(columns={
    'Male': 'mortality_male',
    'Female': 'mortality_female',
    'Total': 'mortality_total'
})

# cleaned dataset
suicide_rate_pivot.head()

In [ ]:
#| output: false
# filtering to keep only the  rows with 'Total' values
suicide_rate_filtered = suicide_rate_subset[suicide_rate_subset["sex"] == "Total"]

suicide_rate_filtered

In [ ]:
#| output: false
# dropping the 'sex' column as it's no longer needed
suicide_rate_filtered = suicide_rate_filtered.drop(columns=["sex"]).sort_values(["country", "year"])

suicide_rate_filtered.head()

In [ ]:
#| output: false
suicide_rate_filtered ["country_code"] = cc.convert(suicide_rate_filtered ["country"], to="ISO3")

# Verify the result
print(suicide_rate_filtered .head())
print(f"New shape of the DataFrame: {suicide_rate_filtered.shape}")

In [ ]:
#| output: false
print(happiness_data_long.shape)
print(internet_use_subset.shape)
print(suicide_rate_filtered.shape)
print(suicide_rate_pivot.shape)

In [ ]:
#| output: false
# joining suicide_rate with happiness on 'country_code' and 'year'
data_frames = [suicide_rate_filtered, 
    happiness_data_long, internet_use_subset]
df_merged = reduce(lambda left, right: pd.merge(left, right,  on=['country_code','year'],
    how='outer'), data_frames)

# replacing missing values
# replacing NaN with 'Unknown' for object (string) columns
for col in df_merged.select_dtypes(include='object').columns:
    df_merged[col].fillna('Unknown', inplace=True)

# replacing NaN with -1 for numeric columns
for col in df_merged.select_dtypes(include='number').columns:
    df_merged[col].fillna(-1, inplace=True)

df_merged.head(20)

In [ ]:
#| output: false
# dropping unnecessary columns
df_merged = df_merged.drop(columns=["country_x", "country"])
df_merged.rename(columns={"country_y": "country"}, inplace=True)
df_merged.head(20)

In [ ]:
#| output: false
# rearranging columns to a more intuitive order
column_order = ['country_code', 'country', 'year', 'percent_happiness', 'mortality_count', 'percent_internet_household']
if set(column_order).issubset(df_merged.columns):
    df_merged = df_merged[column_order]

# Check the data types
df_merged

In [ ]:
#| output: false
df_merged.to_csv('final_merged.csv', index=False)

# Rising and Falling Joy: 5-Year Happiness Trends

## {height = "10%"}


In [ ]:
# 1. Global Average Happiness
global_avg_happiness = happiness_data_long["percent_happiness"].mean()
print(f"Global happiness averaged {global_avg_happiness:.1f}%.")

# 2. Happiest Year Globally
happiest_year = (
    happiness_data_long.groupby("year")["percent_happiness"].mean()
    .idxmax()
)
happiest_year_avg = (
    happiness_data_long.groupby("year")["percent_happiness"].mean()
    .max()
)
print(f"The happiest year globally was {happiest_year}, with an average happiness of {happiest_year_avg:.1f}%.")

# 3. Most Stable Performer
country_variability = (
    happiness_data_long.groupby("country")["percent_happiness"].std()
)
most_stable_country = country_variability.idxmin()
least_variability = country_variability.min()
print(f"The most stable performer was {most_stable_country}, with a variability of ±{least_variability:.1f}%.")

# 4. Global Happiness Gap
happiest_country_avg = (
    happiness_data_long.groupby("country")["percent_happiness"].mean()
    .max()
)
least_happy_country_avg = (
    happiness_data_long.groupby("country")["percent_happiness"].mean()
    .min()
)
happiness_gap = happiest_country_avg - least_happy_country_avg
print(f"The global happiness gap is {happiness_gap:.1f} percentage points.")

::: {.valuebox title="Countries Analyzed" color="#d4af37"}
`{python} happy_countries`
:::

::: {.valuebox title="Global Avg Happiness" color="#d4af37"}
`{python} str(global_avg_happiness.round(1)`
:::

::: {.valuebox title="Happiest Year" color="primary"}
`{python} str(happiest_year)`
:::

::: {.valuebox title="Happiest Country Avg" color="primary"}
`{python} str(happiest_country_avg.round(1))`
:::


::: {.valuebox title="Happiness Gap" color="primary"}
`{python} str(happiness_gap.round(1))`
:::

## Row {height="45%"}

::: {.card title="From Blue to Gold: Track of Global Happiness Among the Top 10 Countries"}


In [ ]:
# Find the top 10 countries overall based on maximum percent_happiness
top_countries_overall = df_merged.groupby("country")["percent_happiness"].max().nlargest(10).index

# Filter the data to include only these countries across all years
consistent_top_countries = df_merged[df_merged["country"].isin(top_countries_overall)]

# Create a bar chart
fig = px.bar(
    consistent_top_countries,
    x="country",
    y="percent_happiness",
    color="year",
    labels={"percent_happiness": "Percent Happiness", "country": "Country"},
    text="percent_happiness"
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(
    xaxis_title="Country",
    yaxis_title="Percent Happiness",
    showlegend=True
)

fig.show()


# Calculate the highest scoring country and its score
highest_country = consistent_top_countries.groupby("country")["percent_happiness"].max().idxmax()
highest_score = consistent_top_countries.groupby("country")["percent_happiness"].max().max()

# Calculate Canada's ranking
country_scores = consistent_top_countries.groupby("country")["percent_happiness"].max()
ranked_countries = country_scores.sort_values(ascending=False).reset_index()
canada_rank = ranked_countries[ranked_countries["country"] == "Canada"].index[0] + 1
canada_score = country_scores["Canada"]

# Output in two sentences
output = (
    f"For 5 Years Straight, {highest_country} Tops the Global Happiness Chart at {highest_score:.1f}% while Canada Proudly Ranks {canada_rank}th at {canada_score:.1f}%"
)
print(output)

:::

## Row  {.tabset height="45%"}
::: {.card title="Global Happiness Unveiled: 5-Year Journey of Growth, Crisis, and Decline"}


In [ ]:
#| label: Global Happiness Trend

# Calculate global average happiness per year
global_trends = happiness_data_long.groupby("year")["percent_happiness"].mean().reset_index()

# Create a line chart
line_fig = px.line(
    global_trends,
    x="year",
    y="percent_happiness",
    title="Global Average Happiness Trends (Yearly)",
    labels={"percent_happiness": "Average Happiness (%)", "year": "Year"},
    markers=True
)

# Annotating key event: pandemic in 2020)
line_fig.add_annotation(
    x=2020,
    y=global_trends.loc[global_trends["year"] == 2020, "percent_happiness"].values[0],
    text="Pandemic Impact",
    showarrow=True,
    arrowhead=2,
    ax=-40,
    ay=-40
)

# Show the chart
line_fig.show()

:::

::: {.card title="Insight"}
This line chart visualizes the global average happiness trend from 2017 to 2022. It shows a steady rise in happiness from 2017 to 2020, peaking during the pandemic year at 57.5%. While this might seem counterintuitive, it reflects resilience in high-ranking nations like Finland and Denmark, which helped maintain high global averages. After 2020, a noticeable decline highlights the longer-term impacts of the pandemic, economic challenges, and global uncertainty. This visual emphasizes how global averages can mask country-specific variations, making it essential to analyze individual trends for a deeper understanding.
:::
